In [19]:
import pandas as pd
import requests
from pandas.io.json import json_normalize
import json
import csv
import bar_chart_race as bcr
import matplotlib as plt 

In [20]:
pd.set_option("display.max_rows", None, "display.max_columns", None) # to see the complete DF and not the truncated version

In [21]:
league_url ='https://fantasy.premierleague.com/api/leagues-classic/1771282/standings/'
r = requests.get(league_url).text
json_data = json.loads(r)
# print (json.dumps(json_data, indent=2))

In [22]:
#converting to DF

In [23]:
league_data = pd.DataFrame(json_data['standings']['results'])

In [24]:
#creating list of dataframes for each individual player 

In [25]:
all_dataframes = []
player_id = league_data['entry'].tolist()
all_name = iter(league_data["player_name"].tolist())
no_gw = 9 # total no. of GW for the league

for team in player_id:

    team_url = 'https://fantasy.premierleague.com/api/entry/' + str(team) + '/history/' #inidividual address for each player
    p_id = requests.get(team_url).text
    teams_json_data = json.loads(p_id)
#     print(json.dumps(teams_json_data, indent =2))  #prints JSON for every player

    dataframe = pd.DataFrame(teams_json_data['current'], columns = ["total_points"])  #create dataframes for each player       
#     dataframe = pd.DataFrame(row, columns = ["GW", "total_points"])

    # addressing for player with old accounts 
    if len(dataframe['total_points']) >= 20:
        dataframe = dataframe.diff()
        dataframe = dataframe.iloc[-7:] #update this every GW
        dataframe = dataframe.cumsum()
        
        
                
    dataframe = dataframe.rename(columns = {"total_points": next(all_name)})
    dataframe = dataframe.reset_index(drop=True)
    all_dataframes.append(dataframe)

In [26]:
final_dfs = pd.concat(all_dataframes, axis=1)

In [27]:
# Add GW columns here
GW = ["30+","31+","32+", "33+","34+","35+","36+"] #update this every GW
final_dfs.insert (0, "GW", GW)

In [28]:
final_dfs = final_dfs.set_index("GW")

In [29]:
final_dfs

,Mubtasim Fuad,Mahbube Rabbani,shareque zaman,Saraf Hasan,Pial Rahman,Tasnimul Hasan,Tauhid Sefo,Likhon Amin,Abrar Hossain,Asif Bin Zaman,Muntasir Azad,Mahedur Rahman,Zabir Sami
GW,,,,,,,,,,,,,
30+,69,61,95,72,92,59,66.0,64.0,56,61,55.0,77.0,73
31+,137,124,164,135,148,114,140.0,113.0,125,113,121.0,151.0,148
32+,230,187,260,220,209,199,196.0,190.0,223,162,176.0,204.0,184
33+,286,259,311,288,259,259,267.0,245.0,277,228,233.0,246.0,240
34+,347,357,371,346,329,343,336.0,318.0,327,302,289.0,304.0,291
35+,400,409,403,397,394,395,378.0,372.0,358,347,351.0,359.0,334
36+,418,411,403,400,398,395,374.0,374.0,364,354,353.0,351.0,337


In [30]:
# Visuualization

In [32]:

bcr.bar_chart_race(
    df=final_dfs,
    filename= None,
    orientation='h',
    sort='desc',
    n_bars=13,
    fixed_order=False,
    fixed_max=True,
    steps_per_period=85,
    interpolate_period=False,
    label_bars=True,
    bar_size=.95,
    period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center'},
    period_fmt='GW {x}',
    period_summary_func=lambda v, r: {'x': .99, 'y': .38,
                                      's': f'Highest point in GW: {v.nlargest(13).max():,.0f}',
                                      'ha': 'right', 'size': 8, 'family': 'Courier New'},
    perpendicular_bar_func='median',
    period_length=6500,
    figsize=(5, 3),
    dpi=144,
    cmap='dark12',
    title='Nuno Nothing Jon Snow Dancing Chair',
    title_size='',
    bar_label_size=7,
    tick_label_size=7,
    shared_fontdict={'family' : 'Helvetica', 'color' : '.1'},
    scale='linear',
    writer=None,
    fig=None,
    bar_kwargs={'alpha': .7},
    filter_column_colors=False)  